<a href="https://colab.research.google.com/github/robertjprior/CausaLM/blob/master/Mistral_7B_Instruct_Training_for_Text_Summarisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import torch
# if str(torch.__version__) != "2.1.1+cu121"
#   print(torch.__version__)
# # 1.13.0+cu116

#   !pip3 install numpy --pre torch[dynamo] --force-reinstall --extra-index-url https://download.pytorch.org/whl/nightly/cu117


#   # Click on RESTART RUNTIME

# import torch
# print(torch.__version__)
# # 1.14.0.dev20221207+cu117

In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes xformers einops
!pip -q install langchain
!pip install git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git



  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Operation cancelled by user


In [ ]:
#!pip install transformers==4.30

In [ ]:
from datasets import Dataset
from pathlib import Path
import datasets

In [ ]:
import os, sys
from pathlib import Path
from google.colab import drive

nb_path = 'generated_training_data'
try:
  drive.mount('/content/drive')
  #mounting: https://stackoverflow.com/questions/55253498/how-do-i-install-a-library-permanently-in-colab

  os.symlink('/content/drive/MyDrive/Colab Notebooks/MistralSummarization', nb_path)

  sys.path.insert(0,nb_path)
except FileExistsError:
  pass

In [ ]:
!nvidia-smi

## Mistral 7B Instruct



In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
#torch.set_default_device('cuda')

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    #load_in_8bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("filipealmeida/Mistral-7B-v0.1-sharded")

model = AutoModelForCausalLM.from_pretrained("filipealmeida/Mistral-7B-v0.1-sharded", quantization_config=bnb_config, trust_remote_code=True, device_map='auto')
                                          #torch_dtype="auto")



In [ ]:
# text = "<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST]"

# encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

# device = 'cuda'
# model_inputs = encodeds.to(device)
# #model.to(device)

# generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
# decoded = tokenizer.batch_decode(generated_ids)
# print(decoded[0])

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
print(model)

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"], #["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

In [ ]:
print(model)

In [ ]:
#import peft
#model.add_adapter(peft_config = config, adapter_name = "adapter1")

In [ ]:
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

# Prep Summarization Dataset for training
dataset: https://github.com/Alex-Fabbri/Multi-News

In [ ]:
training_dataset_size = 200

In [ ]:
file = open(Path(nb_path, 'test.txt.src.tokenized.fixed.cleaned.final.truncated')).read()
articles = file.split('\n')[0:training_dataset_size]
file = open(Path(nb_path, 'test.txt.tgt.tokenized.fixed.cleaned.final.truncated')).read()
target = file.split('\n')[0:training_dataset_size]

print(len(articles))
print(len(target))

In [ ]:
def formatting_prompts_func_batch(articles, summary):
    output_texts = []
    for i in range(len(articles)):
        #text = f"### Question: {example['prompt'][i]}\n ### Answer: {example['response'][i]}"
        text = f"<s>[INST] Provide a concise summary of the triple backquoted text. '''{articles[i]}'''  Provide the summary in bullet points. [/INST] \\n {summary[i]} </s>"
        output_texts.append(text)
    return output_texts

In [ ]:
train_qa = formatting_prompts_func_batch(articles, target)
train_qa = {"quote":train_qa}

#test_qa = formatting_prompts_func(test_df)
#test_qa = {"quote":test_qa}

In [ ]:
train_qa = Dataset.from_dict(train_qa)
train_qa = train_qa.map(lambda samples: tokenizer(samples["quote"], truncation=True), batched=True)

#test_qa = Dataset.from_dict(test_qa)
#test_qa = test_qa.map(lambda samples: tokenizer(samples["quote"]), batched=True)
train_qa

In [ ]:

data = datasets.DatasetDict({"train":train_qa})#, "test": test_qa})

# Training for Summarization
https://huggingface.co/blog/Lora-for-sequence-classification-with-Roberta-Llama-Mistral#lora

In [ ]:
tokenizer.eos_token

In [ ]:
from torch.utils.data import DataLoader
class CustomTrainer(transformers.Trainer):
  # def __init__(self, **kwargs):
  #   super().__init__(**kwargs)
  def get_train_dataloader(self) -> DataLoader:
        """
        Returns the training [`~torch.utils.data.DataLoader`].

        Will use no sampler if `train_dataset` does not implement `__len__`, a random sampler (adapted to distributed
        training if necessary) otherwise.

        Subclass and override this method if you want to inject some custom behavior.
        """


        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")

        train_dataset = self.train_dataset
        data_collator = self.data_collator
        #if is_datasets_available() and isinstance(train_dataset, datasets.Dataset):
        train_dataset = self._remove_unused_columns(train_dataset, description="training")
        #else:
        #    data_collator = self._get_collator_with_removed_columns(data_collator, description="training")

        dataloader_params = {
            "batch_size": self._train_batch_size,
            "collate_fn": data_collator,
            "num_workers": self.args.dataloader_num_workers,
            "pin_memory": self.args.dataloader_pin_memory,
            #"generator": torch.Generator('cuda'),
            #"shuffle": False
        }

        if not isinstance(train_dataset, torch.utils.data.IterableDataset):
            dataloader_params["sampler"] = self._get_train_sampler()
            dataloader_params["drop_last"] = self.args.dataloader_drop_last
            #dataloader_params["worker_init_fn"] = seed_worker

        return self.accelerator.prepare(DataLoader(train_dataset, shuffle=False, **dataloader_params))


In [ ]:
trainer = transformers.Trainer( #CustomTrainer(
    model=model,
    train_dataset=data["train"],
    #eval_dataset = data["test"],
    args=transformers.TrainingArguments(
        gradient_checkpointing=True,
        per_device_train_batch_size=3,
        gradient_accumulation_steps=4,
        num_train_epochs=20, #3
        warmup_steps=50,
        #lr_scheduler_type = 'linear',
        max_steps=-1,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir=Path(nb_path), #"outputs",
        optim="paged_adamw_8bit",
        save_strategy = "epoch",
        save_total_limit=1,
        #evaluation_strategy = 'epoch',
        #metric_for_best_model, -defaults to loss, otherwise function returned from eval
        #greater_is_better, - used in conjunction with above
        #
        #compute_metrics=compute_metrics,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


In [30]:
#import math
#eval_results_baseline = trainer.evaluate(data['test'])
#print(f"Perplexity: {math.exp(eval_results_baseline['eval_loss']):.2f}")

In [31]:
#old_collator = trainer.data_collator
#trainer.data_collator = lambda data: dict(old_collator(data))

In [32]:
#!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'

In [35]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.626100
2,1.596000
3,1.605000
4,1.679000
5,1.543200
6,1.550600
7,1.547900
8,1.541100
9,1.572500
10,1.603900


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

KeyboardInterrupt: ignored

### Now move over to the Summarization Tests file to evaluate the trained model